In [37]:
#imports
import pandas as pd
import numpy as np
import os
import psycopg2
from decimal import Decimal
from dotenv import load_dotenv
from functions import sql_to_dataframe, connect, fetch_column_names
load_dotenv()
from sqlalchemy import create_engine, inspect
import matplotlib.pyplot as plt

from importlib import reload

from lib import masking_functions, relative_entropy_copy, relative_entropy_estimator
reload(masking_functions)
reload(relative_entropy_copy)
reload(relative_entropy_estimator)

from lib.masking_functions import mask, noise_date, generalize_date, generalize_number, suppress, bucketize, inverse_generalize_date_SO, inverse_generalize_date, inverse_bucketize_SO, add_relative_noise, inverse_bucketize, FloatAlphabet, DateAlphabet, inverse_blur_string, inverse_bucketize, inverse_generalize_number, blur_string, inverse_suppress_phone, inverse_blur_phone, inverse_suppress_segment, inverse_generalize_number_SO, inverse_noise_date, inverse_noise_date_SO
from lib.relative_entropy_copy import compute_prob_dist, specification_prob, relative_entropy
from lib.relative_entropy_estimator import DB_stats, estimate_rel_entropy, convert_to_array, get_stat

/Users/avanedaei/Desktop/thesis/thesis/thesis/lib/relative_entropy_estimator.py:174: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  # if self.n_distinct != self.dist.size:


In [2]:
from datetime import datetime, timedelta

def generate_date_range(start_date_str, end_date_str):
    """
    Generate all dates between two given dates (inclusive).

    Args:
        start_date_str (str): The start date in "YYYY-MM-DD" format.
        end_date_str (str): The end date in "YYYY-MM-DD" format.

    Returns:
        list: A list of dates as strings in "YYYY-MM-DD" format.
    """
    try:
        # Convert string dates to datetime objects
        start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
        end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

        # Check if the start date is before or the same as the end date
        if start_date > end_date:
            raise ValueError("The start date must be before or the same as the end date.")

        # Generate the list of dates
        current_date = start_date
        date_list = []
        while current_date <= end_date:
            date_list.append(current_date.strftime("%Y-%m-%d"))
            current_date += timedelta(days=1)  # Move to the next day

        return date_list
    
    except ValueError as e:
        raise ValueError(f"Date error: {e}")

# Example usage
start_date = "1990-01-01"
end_date = "2000-01-01"
all_dates = generate_date_range(start_date, end_date)

# DB connection

In [3]:
# Load environment variables from .env file
load_dotenv(dotenv_path='/Users/avanedaei/Desktop/thesis/main/thesis/touch.env')

True

## Opening Connection

In [4]:
# Fetch connection details from environment variables
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')

db_url = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

try:
    engine = create_engine(db_url)
    connection = engine.connect()
    print("Connection successful")
except Exception as e:
    print(f"Error connecting to the database: {e}")

Connection successful


## Customer Dataset

In [5]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_original = pd.read_sql(query, connection)
    if customer_original.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_original.head()

,c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment
0,1,Customer#000000001,"IVhzIApeRb ot,c,E",15,25-989-741-2988,711.56,BUILDING,"to the even, regular platelets. regular, ironi..."
1,2,Customer#000000002,"XSTf4,NCwDVaWNe6tEgvwfmRchLXak",13,23-768-687-3665,121.65,AUTOMOBILE,l accounts. blithely ironic theodolites integr...
2,3,Customer#000000003,MG9kdTD2WBHm,1,11-719-748-3364,7498.12,AUTOMOBILE,"deposits eat slyly ironic, even instructions...."
3,4,Customer#000000004,XxVSJsLAGtn,4,14-128-190-5944,2866.83,MACHINERY,"requests. final, regular ideas sleep final accou"
4,5,Customer#000000005,KvpyuHCplrB84WgAiGV6sYpZq7Tj,3,13-750-942-6364,794.47,HOUSEHOLD,n accounts will have to unwind. foxes cajole a...


In [6]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_p3'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_p3_masked = pd.read_sql(query, connection)
    if customer_p3_masked.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_p3_masked.head()

,c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment
0,1,Customer#000000001,"IVhzIApeRb ot,c,E",15,25-989-741-2988,700.0,BUILDING,"to the even, regular platelets. regular, ironi..."
1,2,Customer#000000002,"XSTf4,NCwDVaWNe6tEgvwfmRchLXak",13,23-768-687-3665,100.0,AUTOMOBILE,l accounts. blithely ironic theodolites integr...
2,3,Customer#000000003,MG9kdTD2WBHm,1,11-719-748-3364,7400.0,AUTOMOBILE,"deposits eat slyly ironic, even instructions...."
3,4,Customer#000000004,XxVSJsLAGtn,4,14-128-190-5944,2800.0,MACHINERY,"requests. final, regular ideas sleep final accou"
4,5,Customer#000000005,KvpyuHCplrB84WgAiGV6sYpZq7Tj,3,13-750-942-6364,700.0,HOUSEHOLD,n accounts will have to unwind. foxes cajole a...


In [7]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_p1'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_p1_masked = pd.read_sql(query, connection)
    if customer_p1_masked.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_p1_masked.head()

,c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment
0,1,Customer#000000001,"IVhzIApeRb ot,c,E",15,25-989-741-2988,711.56,XXXXX,"to the even, regular platelets. regular, ironi..."
1,2,Customer#000000002,"XSTf4,NCwDVaWNe6tEgvwfmRchLXak",13,23-768-687-3665,121.65,XXXXX,l accounts. blithely ironic theodolites integr...
2,3,Customer#000000003,MG9kdTD2WBHm,1,11-719-748-3364,7498.12,XXXXX,"deposits eat slyly ironic, even instructions...."
3,4,Customer#000000004,XxVSJsLAGtn,4,14-128-190-5944,2866.83,XXXXX,"requests. final, regular ideas sleep final accou"
4,5,Customer#000000005,KvpyuHCplrB84WgAiGV6sYpZq7Tj,3,13-750-942-6364,794.47,XXXXX,n accounts will have to unwind. foxes cajole a...


In [8]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_p2_limited'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_p2_masked = pd.read_sql(query, connection)
    if customer_p2_masked.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_p2_masked.head()

,c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment
0,1,Customer#000000001,"IVhzIApeRb ot,c,E",15,25-989-741-2988,500.0,BUILDING,"to the even, regular platelets. regular, ironi..."
1,2,Customer#000000002,"XSTf4,NCwDVaWNe6tEgvwfmRchLXak",13,23-768-687-3665,0.0,AUTOMOBILE,l accounts. blithely ironic theodolites integr...
2,3,Customer#000000003,MG9kdTD2WBHm,1,11-719-748-3364,7250.0,AUTOMOBILE,"deposits eat slyly ironic, even instructions...."
3,4,Customer#000000004,XxVSJsLAGtn,4,14-128-190-5944,2750.0,MACHINERY,"requests. final, regular ideas sleep final accou"
4,5,Customer#000000005,KvpyuHCplrB84WgAiGV6sYpZq7Tj,3,13-750-942-6364,750.0,HOUSEHOLD,n accounts will have to unwind. foxes cajole a...


In [9]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_stat'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_stats_original = pd.read_sql(query, connection)
    if customer_stats_original.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_stats_original.head(6)

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,c_custkey,-1.00000,None,None,"[4, 1438, 2918, 4610, 6042, 7405, 8937, 10403,..."
1,c_name,-1.00000,None,None,"[Customer#000000004, Customer#000001438, Custo..."
2,c_address,-1.00000,None,None,"[ 3Q UJOXDtbuQTGqf, c3rGrkl c, ,JP9CrKtS49ye..."
3,c_nationkey,25.00000,"[20, 19, 4, 16, 1, 0, 5, 12, 22, 2, 15, 11, 24...","[0.0425, 0.0415, 0.04146667, 0.040933333, 0.04...",None
4,c_phone,-1.00000,None,None,"[10-100-220-4520, 10-302-593-3280, 10-526-919-..."
5,c_acctbal,-0.89094,None,None,"[-999.99, -894.27, -792.60, -688.86, -572.50, ..."


In [10]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_p3_stat'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_p3_stats_masked = pd.read_sql(query, connection)
    if customer_p3_stats_masked.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_p3_stats_masked.head()

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,c_custkey,-1.0,None,None,"[8, 1523, 2886, 4333, 5781, 7256, 8815, 10476,..."
1,c_name,-1.0,None,None,"[Customer#000000008, Customer#000001523, Custo..."
2,c_address,-1.0,None,None,"[ 2uZwVhQvwA, e6yYA7oh54d, ,LEzibjYHgo9Nles..."
3,c_nationkey,25.0,"[3, 18, 2, 21, 11, 4, 16, 12, 9, 23, 14, 0, 8,...","[0.042366665, 0.0418, 0.041733332, 0.0415, 0.0...",None
4,c_phone,-1.0,None,None,"[10-100-106-1617, 10-316-300-2869, 10-552-487-..."


In [11]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_p1_stat'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_p1_stats_masked = pd.read_sql(query, connection)
    if customer_p1_stats_masked.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_p1_stats_masked.head(6)

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,c_custkey,-1.000000,None,None,"[1, 1633, 3132, 4594, 6107, 7623, 9005, 10526,..."
1,c_name,-1.000000,None,None,"[Customer#000000001, Customer#000001633, Custo..."
2,c_address,-1.000000,None,None,"[ 821GWWou3sOyp,, e81tT4AnjhKv9l9Z, ,HVvSY2..."
3,c_nationkey,25.000000,"[22, 6, 0, 13, 3, 12, 9, 5, 4, 15, 19, 17, 24,...","[0.0429, 0.0426, 0.040933333, 0.0409, 0.040733...",None
4,c_phone,-1.000000,None,None,"[10-101-751-3934, 10-337-291-3047, 10-548-585-..."
5,c_acctbal,-0.892873,None,None,"[-999.99, -885.20, -778.56, -656.75, -549.10, ..."


In [12]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_p2_limited_stats'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_p2_stats_masked = pd.read_sql(query, connection)
    if customer_p2_stats_masked.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_p2_stats_masked.head(6)

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,c_custkey,-1.00,None,None,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,c_name,-1.00,None,None,"[Customer#000000001, Customer#000000002, Custo..."
2,c_address,-1.00,None,None,"[0UU iPhBupFvemNB, 3TvCzjuPzpJ0,DdJ8kW5U, 3txG..."
3,c_nationkey,-0.42,"[3, 8, 1, 2, 6, 10, 13, 15, 17, 23, 0, 5, 12, ...","[0.12, 0.08, 0.06, 0.06, 0.06, 0.06, 0.06, 0.0...","[4, 9, 16, 19, 20, 21]"
4,c_phone,-1.00,None,None,"[10-508-348-5882, 10-773-203-7342, 11-719-748-..."
5,c_acctbal,-0.58,"[2750.0, 3250.0, 7500.0, 0.0, 250.0, 500.0, 10...","[0.06, 0.06, 0.06, 0.04, 0.04, 0.04, 0.04, 0.0...","[-1000.0, -500.0, -250.0, 750.0, 4250.0, 4750...."


In [13]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_p4'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_p4_masked = pd.read_sql(query, connection)
    if customer_p4_masked.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_p4_masked.head(6)

,c_custkey,c_name,c_address,c_nationkey,c_phone,c_acctbal,c_mktsegment,c_comment
0,1,Customer#000000001,"IVhzIApeRb ot,c,E",15,25-989-741-2XXX,711.56,BUILDING,"to the even, regular platelets. regular, ironi..."
1,2,Customer#000000002,"XSTf4,NCwDVaWNe6tEgvwfmRchLXak",13,23-768-687-3XXX,121.65,AUTOMOBILE,l accounts. blithely ironic theodolites integr...
2,3,Customer#000000003,MG9kdTD2WBHm,1,11-719-748-3XXX,7498.12,AUTOMOBILE,"deposits eat slyly ironic, even instructions...."
3,4,Customer#000000004,XxVSJsLAGtn,4,14-128-190-5XXX,2866.83,MACHINERY,"requests. final, regular ideas sleep final accou"
4,5,Customer#000000005,KvpyuHCplrB84WgAiGV6sYpZq7Tj,3,13-750-942-6XXX,794.47,HOUSEHOLD,n accounts will have to unwind. foxes cajole a...
5,6,Customer#000000006,"sKZz0CsnMD7mp4Xd0YrBvx,LREYKUWAh yVn",20,30-114-968-4XXX,7638.57,AUTOMOBILE,tions. even deposits boost according to the sl...


In [14]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'c_p4_stat'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    customer_p4_stats_masked = pd.read_sql(query, connection)
    if customer_p4_stats_masked.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

customer_p4_stats_masked.head(6)

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,c_acctbal,-0.887053,None,None,"[-999.99, -900.40, -794.22, -686.99, -566.41, ..."
1,c_mktsegment,5.000000,"[FURNITURE , HOUSEHOLD , BUILDING , MACHINERY...","[0.2049, 0.2018, 0.1993, 0.19893333, 0.19506666]",None
2,c_comment,-0.999100,None,None,[ Tiresias along the blithely regular packages...
3,c_custkey,-1.000000,None,None,"[4, 1627, 3023, 4504, 6036, 7531, 9072, 10678,..."
4,c_name,-1.000000,None,None,"[Customer#000000004, Customer#000001627, Custo..."
5,c_address,-1.000000,None,None,"[ 821GWWou3sOyp,, aVmqqzD4RvHEQV0, ,FrhVmSN..."


## Order Dataset

In [15]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'o'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    orders_original = pd.read_sql(query, connection)
    if orders_original.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

orders_original.head()

,o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
0,1,36901,O,173665.47,1996-01-02,5-LOW,Clerk#000000951,0,nstructions sleep furiously among
1,2,78002,O,46929.18,1996-12-01,1-URGENT,Clerk#000000880,0,"foxes. pending accounts at the pending, silen..."
2,3,123314,F,193846.25,1993-10-14,5-LOW,Clerk#000000955,0,sly final accounts boost. carefully regular id...
3,4,136777,O,32151.78,1995-10-11,5-LOW,Clerk#000000124,0,"sits. slyly regular warthogs cajole. regular, ..."
4,5,44485,F,144659.20,1994-07-30,5-LOW,Clerk#000000925,0,quickly. bold deposits sleep slyly. packages u...


In [16]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'o_p2'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    orders_masked_p2 = pd.read_sql(query, connection)
    if orders_masked_p2.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

orders_masked_p2.head()

,o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
0,1,36901,O,173665.47,1996-01-01,5-LOW,Clerk#000000951,0,nstructions sleep furiously among
1,2,78002,O,46929.18,1996-12-01,1-URGENT,Clerk#000000880,0,"foxes. pending accounts at the pending, silen..."
2,3,123314,F,193846.25,1993-10-01,5-LOW,Clerk#000000955,0,sly final accounts boost. carefully regular id...
3,4,136777,O,32151.78,1995-10-01,5-LOW,Clerk#000000124,0,"sits. slyly regular warthogs cajole. regular, ..."
4,5,44485,F,144659.20,1994-07-01,5-LOW,Clerk#000000925,0,quickly. bold deposits sleep slyly. packages u...


In [17]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'o_p1'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    orders_masked_p1 = pd.read_sql(query, connection)
    if orders_masked_p1.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

orders_masked_p1.head()

,o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
0,1,36901,O,173665.47,1995-12-23,5-LOW,Clerk#000000951,0,nstructions sleep furiously among
1,2,78002,O,46929.18,1996-11-25,1-URGENT,Clerk#000000880,0,"foxes. pending accounts at the pending, silen..."
2,3,123314,F,193846.25,1993-10-06,5-LOW,Clerk#000000955,0,sly final accounts boost. carefully regular id...
3,4,136777,O,32151.78,1995-10-20,5-LOW,Clerk#000000124,0,"sits. slyly regular warthogs cajole. regular, ..."
4,5,44485,F,144659.20,1994-07-27,5-LOW,Clerk#000000925,0,quickly. bold deposits sleep slyly. packages u...


In [18]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'o_stat'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    orders_stats_original = pd.read_sql(query, connection)
    if orders_stats_original.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

orders_stats_original.head()

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,o_orderkey,-1.000000,None,None,"[34, 57284, 117574, 177600, 234790, 291392, 34..."
1,o_custkey,85643.000000,None,None,"[16, 1468, 2912, 4394, 5839, 7450, 8887, 10399..."
2,o_orderstatus,3.000000,"[O, F, P]","[0.48843333, 0.48576668, 0.0258]",None
3,o_totalprice,-0.961894,None,None,"[927.91, 5838.56, 10936.78, 15365.87, 19403.67..."
4,o_orderdate,2406.000000,"[1997-01-09, 1992-04-30, 1997-07-28, 1997-10-0...","[0.00086666667, 0.00083333335, 0.00083333335, ...","[1992-01-01, 1992-01-26, 1992-02-20, 1992-03-1..."


In [19]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'o_p2_stat'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    orders_stats_masked_p2 = pd.read_sql(query, connection)
    if orders_stats_masked_p2.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

orders_stats_masked_p2.head()

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,o_orderkey,-1.000000,None,None,"[324, 59395, 115174, 173478, 241733, 299364, 3..."
1,o_custkey,87717.000000,None,None,"[37, 1582, 3130, 4588, 6109, 7630, 9316, 10690..."
2,o_orderstatus,3.000000,"[F, O, P]","[0.4883, 0.4865, 0.0252]",None
3,o_totalprice,-0.952812,None,None,"[952.90, 5646.13, 10082.51, 14335.16, 18547.70..."
4,o_orderdate,80.000000,"[1998-07-01, 1994-03-01, 1994-01-01, 1997-05-0...","[0.014366667, 0.013966667, 0.013933334, 0.0138...",None


In [20]:
#creating a query variable to store our query to pass into the function
# Specify the table name from ['customer','lineitem','nation','orders','part','partsupp','region','supplier']
table_name = 'o_p1_stat'
# Fetch column names dynamically
# column_names = fetch_column_names(conn, table_name)
# column_names_str = ', '.join(column_names)
inspector = inspect(engine)
columns = inspector.get_columns(table_name)
column_names = [column['name'] for column in columns]
column_names_str = ', '.join(column_names)
# Construct the SQL query dynamically
try:
    # query = f"SELECT {column_names_str} FROM {table_name}"
    query = f"SELECT * FROM {table_name}"

    orders_stats_masked_p1 = pd.read_sql(query, connection)
    if orders_stats_masked_p1.empty:
        print("Query returned no results.")
    else:
        pass
except Exception as e:
    print(f"Error executing query: {e}")
#loading our dataframe

orders_stats_masked_p1.head()

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,o_orderkey,-1.000000,None,None,"[36, 58371, 116224, 179558, 239268, 299910, 36..."
1,o_custkey,85910.000000,None,None,"[8, 1591, 2938, 4447, 5983, 7547, 9058, 10610,..."
2,o_orderstatus,3.000000,"[O, F, P]","[0.49116668, 0.48346666, 0.025366666]",None
3,o_totalprice,-0.946851,None,None,"[884.52, 5557.24, 9959.78, 14203.77, 18033.36,..."
4,o_orderdate,2423.000000,"[1996-03-14, 1996-11-30, 1993-12-17, 1998-07-0...","[0.0009, 0.0009, 0.00086666667, 0.00086666667,...","[1991-12-22, 1992-01-25, 1992-02-17, 1992-03-1..."


## Closing Connection

In [21]:
#closing the connection
try:
    connection.close()
    print("Connection closed")
except Exception as e:
    print(f"Error closing the connection: {e}")
# Let’s see if we loaded the df successfully

Connection closed


# Masking Datasets 
(table needs to be changed in the above section and run again)

### Masking Customer Table

#### suppressing the c_mktsegment column

In [ ]:
# clean up spaces from column c_mktsegment
df['c_mktsegment'] = df['c_mktsegment'].str.replace(r'\s+', '', regex=True)

In [ ]:
customer_m1 = mask(df, suppress, 'c_mktsegment')

In [ ]:
customer_m1.to_csv("/Users/avanedaei/Desktop/thesis/main/thesis/masked/customer/customer_m1.tbl.csv", sep=',', header=False, index=False)
customer_m1.to_csv("/Users/avanedaei/Desktop/thesis/main/thesis/masked/customer/customer_m1.csv", sep=',', header=False, index=False)

#### generalizing the c_acctbal column number

In [ ]:

customer_m2 = mask(df, generalize_number, 'c_acctbal', 100)
customer_m2.head()

In [ ]:
customer_m2.to_csv("/Users/avanedaei/Desktop/thesis/main/thesis/masked/customer/customer_m2.csv", sep=',', header=False, index=False)
customer_m2.to_csv("/Users/avanedaei/Desktop/thesis/main/thesis/masked/customer/customer_m2.tbl.csv", sep=',', header=False, index=False)

#### bucketizing the c_acctbal column number

In [ ]:
customer_m3 = mask(df, bucketize, 'c_acctbal', 500)

In [ ]:
customer_m3.to_csv("/Users/avanedaei/Desktop/thesis/main/thesis/masked/customer/customer_m3.csv", sep=',', header=False, index=False)
customer_m3.to_csv("/Users/avanedaei/Desktop/thesis/main/thesis/masked/customer/customer_m3.tbl.csv", sep=',', header=False, index=False)

In [ ]:
customer_m4 = mask(df, bucketize, 'c_acctbal', 500)

#### generalizing order date based on month

In [ ]:
orders_m2 = mask(orders_original, generalize_date, 'o_orderdate', "MONTH")

In [ ]:
orders_m2.to_csv("/Users/avanedaei/Desktop/thesis/main/thesis/masked/customer/orders_m2.tbl.csv", sep=',', header=False, index=False)
orders_m2.to_csv("/Users/avanedaei/Desktop/thesis/main/thesis/masked/customer/orders_m2.csv", sep=',', header=False, index=False)

# Estimating Relative Entropy

## customer table

### With Masking Functions

##### example: estimating entropy c_mktsegment

In [ ]:
p = compute_prob_dist(df, ['c_mktsegment'])
q = compute_prob_dist(customer_m1, ['c_mktsegment'])

# Convert probability distribution reverting the masking function
# print(q)
new_q = specification_prob(q,inverse_blur_string)
#print(new_q)
row, col = df.shape
size = row * col
relative_entropy(p, new_q,size)

##### example: estimating entropy c_acctbal

In [ ]:
p = compute_prob_dist(df, ['c_acctbal'])
q = compute_prob_dist(customer_m2, ['c_acctbal'])

# Convert probability distribution reverting the masking function
# print(q)
divisor = 100
new_q = specification_prob(q,inverse_generalize_number(divisor))
#print(new_q)
row, col = df.shape
size = row * col
relative_entropy(p, new_q,size)

In [ ]:
# p = compute_prob_dist(df, ['c_acctbal'])
# q = compute_prob_dist(customer_m3, ['c_acctbal'])

# # Convert probability distribution reverting the masking function
# # print(q)
# new_q = specification_prob(q,inverse_bucketize(['c_acctbal'],0.01))
# #print(new_q)
# row, col = df.shape
# size = row * col
# relative_entropy(p, new_q,size)

### With Materialized Views

#### entropy over single column

In [ ]:
p = compute_prob_dist(df, ['c_mktsegment'])
q = compute_prob_dist(materialized_view, ['c_mktsegment'])

# Convert probability distribution reverting the masking function
new_q = specification_prob(q,inverse_suppress_segment)
print(new_q)
row, col = df.shape
size = row * col
relative_entropy(p, new_q,size)


#### entropy over multiple columns

In [ ]:
p = compute_prob_dist(df, ['c_mktsegment', 'c_nationkey'])
q = compute_prob_dist(materialized_view, ['c_mktsegment', 'c_nationkey'])

# Convert probability distribution reverting the masking function
new_q = specification_prob(q,inverse_suppress_segment)
print(new_q)
row, col = df.shape
size = row * col
relative_entropy(p, new_q,size)

In [ ]:
p = compute_prob_dist(df, ['c_mktsegment', 'c_comment'])
q = compute_prob_dist(materialized_view, ['c_mktsegment', 'c_comment'])

# Convert probability distribution reverting the masking function
new_q = specification_prob(q,inverse_suppress_segment)
print(new_q)
row, col = df.shape
size = row * col
relative_entropy(p, new_q,size)

# Estimating Relative Entropy

## c_p4 - blurred c_phone

In [63]:
size = 6001215
# df.head()
customer_stats_original.fillna(np.NaN, inplace=True)
customer_p4_stats_masked.fillna(np.NaN, inplace=True)
# print(customer_stats_original.head())

customer_stats_original["n_distinct"] = customer_stats_original["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))
customer_p4_stats_masked["n_distinct"] = customer_p2_stats_masked["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))


In [39]:
p = compute_prob_dist(customer_original, ['c_phone'])
q = compute_prob_dist(customer_p4_masked, ['c_phone'])

# Convert probability distribution reverting the masking function
# print(q)
new_q = specification_prob(q,inverse_blur_phone)
#print(new_q)
# row, col = customer_original.shape
# size = row * col
# relative_entropy(p, new_q,size)

: 

## c_p2 - generalizing c_acctbal

In [22]:
size = 6001215
# df.head()
customer_stats_original.fillna(np.NaN, inplace=True)
customer_p2_stats_masked.fillna(np.NaN, inplace=True)
# print(customer_stats_original.head())

customer_stats_original["n_distinct"] = customer_stats_original["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))
customer_p2_stats_masked["n_distinct"] = customer_p2_stats_masked["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))


In [23]:
p = compute_prob_dist(customer_original, ['c_acctbal'])
q = compute_prob_dist(customer_p2_masked, ['c_acctbal'])

# Convert probability distribution reverting the masking function
# print(q)
new_q = specification_prob(q,inverse_generalize_number_SO(250))
#print(new_q)
row, col = customer_original.shape
size = row * col
relative_entropy(p, new_q,size)

3.0257280175055947

In [24]:
customer_stats_original.head(6)

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,c_custkey,6001215,NaN,NaN,"[4, 1438, 2918, 4610, 6042, 7405, 8937, 10403,..."
1,c_name,6001215,NaN,NaN,"[Customer#000000004, Customer#000001438, Custo..."
2,c_address,6001215,NaN,NaN,"[ 3Q UJOXDtbuQTGqf, c3rGrkl c, ,JP9CrKtS49ye..."
3,c_nationkey,25,"[20, 19, 4, 16, 1, 0, 5, 12, 22, 2, 15, 11, 24...","[0.0425, 0.0415, 0.04146667, 0.040933333, 0.04...",NaN
4,c_phone,6001215,NaN,NaN,"[10-100-220-4520, 10-302-593-3280, 10-526-919-..."
5,c_acctbal,5346722,NaN,NaN,"[-999.99, -894.27, -792.60, -688.86, -572.50, ..."


In [25]:
customer_p2_stats_masked.head(6)

,attname,n_distinct,most_common_vals,most_common_freqs,histogram_bounds
0,c_custkey,6001215,NaN,NaN,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,c_name,6001215,NaN,NaN,"[Customer#000000001, Customer#000000002, Custo..."
2,c_address,6001215,NaN,NaN,"[0UU iPhBupFvemNB, 3TvCzjuPzpJ0,DdJ8kW5U, 3txG..."
3,c_nationkey,2520510,"[3, 8, 1, 2, 6, 10, 13, 15, 17, 23, 0, 5, 12, ...","[0.12, 0.08, 0.06, 0.06, 0.06, 0.06, 0.06, 0.0...","[4, 9, 16, 19, 20, 21]"
4,c_phone,6001215,NaN,NaN,"[10-508-348-5882, 10-773-203-7342, 11-719-748-..."
5,c_acctbal,3480704,"[2750.0, 3250.0, 7500.0, 0.0, 250.0, 500.0, 10...","[0.06, 0.06, 0.06, 0.04, 0.04, 0.04, 0.04, 0.0...","[-1000.0, -500.0, -250.0, 750.0, 4250.0, 4750...."


In [27]:
# customer_stats_original.loc[customer_stats_original["attname"] == "c_acctbal", "histogram_bounds"] = customer_stats_original[customer_stats_original["attname"] == "c_acctbal"]["histogram_bounds"].apply(lambda x : [float(i) for i in x])

In [38]:
customer_stats_original_db = DB_stats(customer_stats_original, "c_acctbal")
customer_p2_stats_masked_db = DB_stats(customer_p2_stats_masked, "c_acctbal")
customer_p2_stats_masked_db.perturbation()
customer_p2_stats_masked_db.inverse_dist(inverse_generalize_number_SO(250.0,  0.01, "{:.2f}"))

In [48]:
customer_stats_original_db.histogram_bounds

array(['-999.99', '-894.27', '-792.60', '-688.86', '-572.50', '-458.59',
       '-345.84', '-237.74', '-131.57', '-12.97', '93.27', '216.45',
       '319.25', '432.67', '549.54', '655.82', '773.33', '879.77',
       '990.22', '1113.85', '1214.34', '1332.18', '1439.57', '1552.27',
       '1654.98', '1775.39', '1881.68', '1993.64', '2096.39', '2196.73',
       '2306.96', '2405.16', '2514.98', '2615.42', '2718.02', '2824.21',
       '2941.39', '3051.41', '3164.04', '3272.79', '3380.20', '3481.45',
       '3590.72', '3699.60', '3810.30', '3920.76', '4024.28', '4134.18',
       '4242.37', '4360.48', '4473.80', '4582.74', '4696.44', '4811.97',
       '4917.83', '5024.39', '5129.27', '5242.66', '5355.20', '5465.79',
       '5571.44', '5688.86', '5800.32', '5911.24', '6016.93', '6125.36',
       '6237.07', '6365.93', '6480.83', '6587.26', '6691.52', '6798.21',
       '6907.98', '7017.96', '7113.53', '7229.52', '7334.77', '7450.08',
       '7561.78', '7675.46', '7780.47', '7891.93', '8001.20', 

In [39]:
customer_p2_stats_masked_db.histogram_bounds

array(['-1000.0', '-500.0', '-250.0', '750.0', '4250.0', '4750.0',
       '6750.0', '7000.0', '8250.0', '8750.0', '9500.0'], dtype='<U7')

In [40]:
customer_p2_stats_masked_db.inverse_histogram_bounds(inverse_generalize_number_SO(250.0, 0.01, "{:.2f}"))

In [41]:
customer_p2_stats_masked_db.unmsk_histogram_bounds

array(['-1000.00', '-500.00', '-250.00', '750.00', '4250.00', '4750.00',
       '6750.00', '7000.00', '8250.00', '8750.00', '9749.90'], dtype='<U8')

In [53]:
min_value = float(-1000.0)
max_value = float(9999.99)
step = float(0.01)

c_acctbal_alphabet = []
for number in np.arange(min_value, max_value + step, step):
        # Format the number to two decimal places
        # formatted_number = f"{number:.2f}"
        # Combine the letter with the formatted number
        # if str(number).endswith('.0'):
        #         c_acctbal_alphabet.append(f"{number:.1f}")
        # else:
        c_acctbal_alphabet.append(f"{number:.2f}")

In [54]:
len(c_acctbal_alphabet)
print(c_acctbal_alphabet[:6])

['-1000.00', '-999.99', '-999.98', '-999.97', '-999.96', '-999.95']


In [57]:
freq = 0.0

for i, x in enumerate(customer_stats_original_db.hist_apprx_freq):
    freq += x * customer_stats_original_db.hist_apprx_n_distinct[i]

print(freq)

1.0000000000000007


In [56]:
customer_stats_original_db.estimate_hist_freq(c_acctbal_alphabet)

In [55]:
customer_p2_stats_masked_db.estimate_hist_freq(c_acctbal_alphabet, True)

In [58]:
freq = 0.0

for x in customer_p2_stats_masked_db.most_common_freqs:
    freq += x
print(freq)
for i, x in enumerate(customer_p2_stats_masked_db.hist_apprx_freq):
    freq += x * customer_p2_stats_masked_db.hist_apprx_n_distinct[i]

print(freq)

0.7800000000000001
1.0000000000000002


In [60]:
entropy_c_generalize_acctbal_p2 = estimate_rel_entropy(customer_stats_original_db, customer_p2_stats_masked_db)
print(entropy_c_generalize_acctbal_p2)

/Users/avanedaei/Desktop/thesis/thesis/thesis/lib/relative_entropy_estimator.py:265: RuntimeWarning: invalid value encountered in log2
  relative_entropy += original_stats.hist_apprx_n_distinct[ogl_bin_idx] * (approx_p_x*np.log2(approx_p_x/approx_q_x))
/Users/avanedaei/Desktop/thesis/thesis/thesis/lib/relative_entropy_estimator.py:280: RuntimeWarning: invalid value encountered in log2
  relative_entropy += apprx_n_distinct_bin * (approx_p_x*np.log2(approx_p_x/approx_q_x))


nan


## c_mktsegment

In [ ]:
size = 150000
# df.head()
customer_stats_original.fillna(np.NaN, inplace=True)
customer_stats_masked.fillna(np.NaN, inplace=True)
print(customer_stats_masked.head())
customer_stats_original["n_distinct"] = customer_stats_original["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))
customer_stats_masked["n_distinct"] = customer_stats_masked["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))


In [ ]:
# stats_masked.head()
mktsegment_stats_main = DB_stats(customer_stats_original, "c_mktsegment")
mktsegment_stats_masked = DB_stats(customer_stats_masked, "c_mktsegment")
print(mktsegment_stats_main.dist.index)

In [ ]:
mktsegment_stats_masked.inverse_dist(inverse_suppress_segment)
mktsegment_stats_masked.inverse_histogram_bounds(inverse_suppress_segment)
mktsegment_stats_main.estimate_hist_freq(inverse_suppress_segment)
mktsegment_stats_masked.estimate_hist_freq(inverse_suppress_segment)

In [ ]:
estimate_rel_entropy(mktsegment_stats_main, mktsegment_stats_masked)

In [ ]:
stats_masked

In [ ]:
p = compute_prob_dist(df, ['c_mktsegment'])
q = compute_prob_dist(customer_m1, ['c_mktsegment'])

# Convert probability distribution reverting the masking function
# print(q)
new_q = specification_prob(q,inverse_suppress_segment)
#print(new_q)
row, col = df.shape
size = row * col
relative_entropy(p, new_q,size)

In [ ]:
true_ent = 1.9408977646995316
est_ent = 8.279417859324237
np.abs(true_ent-est_ent)/true_ent

In [ ]:
size = 150000
# df.head()
stats_main.fillna(np.NaN, inplace=True)
stats_masked.fillna(np.NaN, inplace=True)
print(stats_masked.head())
stats_main["n_distinct"] = stats_main["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))
stats_masked["n_distinct"] = stats_masked["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))


In [ ]:
# stats_masked.head()
mktsegment_stats_main = DB_stats(stats_main, "c_mktsegment")
mktsegment_stats_masked = DB_stats(stats_masked, "c_mktsegment")
mktsegment_stats_masked
mktsegment_stats_masked.inverse_dist(inverse_suppress_segment)
mktsegment_stats_masked.inverse_histogram_bounds(inverse_suppress_segment)
mktsegment_stats_main.estimate_hist_freq(inverse_suppress_segment)
mktsegment_stats_masked.estimate_hist_freq(inverse_suppress_segment)
print(mktsegment_stats_main.hist_apprx_freq)

In [ ]:
estimate_rel_entropy(mktsegment_stats_main, mktsegment_stats_masked)

In [ ]:
stats_masked

In [ ]:
p = compute_prob_dist(df, ['c_mktsegment'])
q = compute_prob_dist(customer_m1, ['c_mktsegment'])

# Convert probability distribution reverting the masking function
# print(q)
new_q = specification_prob(q,inverse_suppress_segment)
#print(new_q)
row, col = df.shape
size = row * col
relative_entropy(p, new_q,size)

In [ ]:
true_ent = 1.9408977646995316
est_ent = 8.279417859324237
np.abs(true_ent-est_ent)/true_ent

## order date noise

#### estimating relative entropy

In [ ]:
size = 150000
# df.head()
orders_stats_original.fillna(np.NaN, inplace=True)
orders_stats_masked_p1.fillna(np.NaN, inplace=True)

orders_stats_original["n_distinct"] = orders_stats_original["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))
orders_stats_masked_p1["n_distinct"] = orders_stats_masked_p2["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))

In [ ]:
o_orderdate_stats_original = DB_stats(orders_stats_original, "o_orderdate")

In [ ]:
o_orderdate_stats_p1 = DB_stats(orders_stats_masked_p1, "o_orderdate")

In [ ]:
# o_orderdate_stats_p1.perturbation()
o_orderdate_stats_original.perturbation()
# o_orderdate_stats_p1.inverse_dist(inverse_noise_date_SO("DAYS", 10))
# print(o_orderdate_stats_p1.unmsk_dist)
o_orderdate_stats_p1.inverse_histogram_bounds(inverse_noise_date_SO("DAYS", 10))
# for x in o_orderdate_stats_p1.unmsk_dist:
#     print (x)
# o_orderdate_stats_p1.unmsk_dist
print(o_orderdate_stats_p1.unmsk_dist)
print(o_orderdate_stats_original.unmsk_dist)
# for x in o_orderdate_stats_p1.unmsk_dist['1996-03-10'].keys():
#     print (type(x))


In [ ]:
entropy_o_noise_date_p1 = estimate_rel_entropy(o_orderdate_stats_original, o_orderdate_stats_p1)
print(entropy_o_noise_date_p1)

In [ ]:
print(all_dates)

In [ ]:
o_orderdate_stats_p1.estimate_hist_freq(all_dates)

In [ ]:
o_orderdate_stats_p1.hist_apprx_freq
print(o_orderdate_stats_p1.hist_apprx_freq)

In [ ]:
print(o_orderdate_stats_p1.unmsk_histogram_bounds)

In [ ]:
print(o_orderdate_stats_p1.unmsk_most_common_freqs)
print(o_orderdate_stats_p1.unmsk_most_common_vals)
print(o_orderdate_stats_p1.unmsk_dist)
print(o_orderdate_stats_p1.unmsk_histogram_bounds)

In [ ]:
estimate_rel_entropy(ordersdate_stats_main, ordersdate_stats_masked)

## order date generalization

#### estimating relative entropy

In [47]:
size = 150000
# df.head()
orders_stats_original.fillna(np.NaN, inplace=True)
orders_stats_masked_p2.fillna(np.NaN, inplace=True)

orders_stats_original["n_distinct"] = orders_stats_original["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))
orders_stats_masked_p2["n_distinct"] = orders_stats_masked_p2["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))

In [48]:
from datetime import datetime, timedelta

def generate_date_range(start_date_str, end_date_str):
    """
    Generate all dates between two given dates (inclusive).

    Args:
        start_date_str (str): The start date in "YYYY-MM-DD" format.
        end_date_str (str): The end date in "YYYY-MM-DD" format.

    Returns:
        list: A list of dates as strings in "YYYY-MM-DD" format.
    """
    try:
        # Convert string dates to datetime objects
        start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
        end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

        # Check if the start date is before or the same as the end date
        if start_date > end_date:
            raise ValueError("The start date must be before or the same as the end date.")

        # Generate the list of dates
        current_date = start_date
        date_list = []
        while current_date <= end_date:
            date_list.append(current_date.strftime("%Y-%m-%d"))
            current_date += timedelta(days=1)  # Move to the next day

        return date_list
    
    except ValueError as e:
        raise ValueError(f"Date error: {e}")

# Example usage
start_date = "1992-01-01"
end_date = "1998-08-02"
all_dates = generate_date_range(start_date, end_date)

In [49]:
dates_list = set(list(orders_original["o_orderdate"].map(lambda x: x.strftime('%Y-%m-%d'))))
dates_list_alphabet = list(dates_list)
print(list(dates_list))

['1994-12-21', '1998-02-04', '1993-10-19', '1998-07-06', '1993-08-22', '1998-03-23', '1995-08-18', '1992-07-14', '1995-07-21', '1992-03-14', '1995-01-15', '1993-04-13', '1994-12-10', '1997-08-16', '1998-01-10', '1994-05-02', '1994-03-26', '1993-07-31', '1997-07-22', '1992-03-26', '1993-01-23', '1994-07-21', '1998-06-04', '1992-09-15', '1997-10-19', '1993-05-13', '1998-07-17', '1996-02-25', '1998-03-21', '1994-05-18', '1992-06-15', '1998-05-06', '1996-08-16', '1993-02-12', '1998-02-19', '1997-07-06', '1995-06-20', '1995-01-26', '1994-04-06', '1996-07-21', '1992-06-29', '1995-05-28', '1994-05-22', '1993-06-19', '1998-02-26', '1994-04-14', '1993-11-20', '1992-04-02', '1993-01-21', '1998-07-11', '1993-03-12', '1998-07-30', '1993-08-28', '1994-10-13', '1996-09-05', '1996-07-11', '1993-02-05', '1992-08-12', '1993-01-15', '1993-03-14', '1995-12-20', '1995-11-16', '1995-09-18', '1992-07-08', '1992-06-01', '1994-06-25', '1996-05-24', '1993-06-26', '1997-04-03', '1995-01-18', '1995-09-09', '1992

In [50]:
ordersdate_stats_main = DB_stats(orders_stats_original, "o_orderdate")

In [51]:
ordersdate_stats_masked = DB_stats(orders_stats_masked_p2, "o_orderdate")

In [52]:
ordersdate_stats_masked.perturbation()

In [53]:
# ordersdate_stats_masked.inverse_dist(inverse_generalize_date_SO("MONTH"))
ordersdate_stats_masked.inverse_histogram_bounds(inverse_generalize_date_SO("MONTH"))
# ordersdate_stats_main.estimate_hist_freq(all_dates)
# ordersdate_stats_masked.estimate_hist_freq(all_dates)
# print(ordersdate_stats_main.hist_apprx_freq)

In [57]:
ordersdate_stats_masked.estimate_hist_freq(all_dates, True)
# ordersdate_stats_masked.addapt_freq_selectivity(0.5)

make graph from frequencies

In [59]:
estimate_rel_entropy(ordersdate_stats_main, ordersdate_stats_masked)

-4.908223652361752

#### real relative entropy

In [ ]:
p = compute_prob_dist(orders_original, ['o_orderdate'])
q = compute_prob_dist(orders_masked_p1, ['o_orderdate'])

In [ ]:
print(p)

In [ ]:
print(q)

In [ ]:

new_q = specification_prob(q,inverse_generalize_date_SO("MONTH"))

In [ ]:
print(new_q)

In [ ]:
#print(new_q)
relative_entropy(p, new_q,  orders_masked_p2.size)

In [ ]:
vals = convert_to_array(get_stat(orders_stats_original, "o_orderdate", "most_common_vals"))
freqs = convert_to_array(get_stat(orders_stats_original, "o_orderdate", "most_common_freqs"), True)

p = pd.Series(freqs, index = vals)

vals = convert_to_array(get_stat(orders_stats_masked_p2, "o_orderdate", "most_common_vals"))
freqs = convert_to_array(get_stat(orders_stats_masked_p2, "o_orderdate", "most_common_freqs"), True)

q = pd.Series(freqs, index = vals)

new_q = specification_prob(q,inverse_generalize_date_SO("MONTH"))

relative_entropy(p, new_q, new_q.size)

In [ ]:
real_ent = 0.008263819318169217
est_ent = 0.028624210661386703
np.abs(real_ent-est_ent)/real_ent

#### estimating relative entropy

In [ ]:
size = 150000
# df.head()
orders_stats_original.fillna(np.NaN, inplace=True)
orders_stats_masked_p2.fillna(np.NaN, inplace=True)

orders_stats_original["n_distinct"] = orders_stats_original["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))
orders_stats_masked_p2["n_distinct"] = orders_stats_masked_p2["n_distinct"].apply(lambda x : int(x) if x >= 0 else int(x*-size))

In [ ]:
dates_list = set(list(orders_original["o_orderdate"].map(lambda x: x.strftime('%Y-%m-%d'))))
dates_list_alphabet = list(dates_list)
print(list(dates_list))

In [ ]:
ordersdate_stats_main = DB_stats(orders_stats_original, "o_orderdate")
print(ordersdate_stats_main.histogram_bounds)

In [ ]:
ordersdate_stats_masked = DB_stats(orders_stats_masked_p2, "o_orderdate")
print(ordersdate_stats_masked.histogram_bounds)

In [ ]:
ordersdate_stats_masked.perturbation
ordersdate_stats_masked.inverse_dist(inverse_generalize_date_SO("MONTH"))
ordersdate_stats_masked.inverse_histogram_bounds(inverse_generalize_date_SO("MONTH"))
ordersdate_stats_main.estimate_hist_freq(all_dates)
ordersdate_stats_masked.estimate_hist_freq(all_dates)
print(ordersdate_stats_main.hist_apprx_freq)

In [ ]:
ordersdate_stats_masked.estimate_hist_freq(all_dates, True)

In [ ]:
ordersdate_stats_masked.hist_apprx_freq
print(ordersdate_stats_masked.hist_apprx_freq)

In [ ]:
print(ordersdate_stats_masked.unmsk_histogram_bounds)

In [ ]:
print(ordersdate_stats_masked.unmsk_most_common_freqs)
print(ordersdate_stats_masked.unmsk_most_common_vals)
print(ordersdate_stats_masked.unmsk_dist)
print(ordersdate_stats_masked.unmsk_histogram_bounds)

In [62]:
estimate_rel_entropy(ordersdate_stats_main, ordersdate_stats_masked)

-4.908223652361752

In [ ]:
p = compute_prob_dist(orders_original, ['o_orderdate'])
q = compute_prob_dist(orders_stats_masked_p2, ['o_orderdate'])

In [ ]:
print(p)

In [ ]:
print(q)

In [ ]:

new_q = specification_prob(q,inverse_generalize_date_SO("MONTH"))

In [ ]:
print(new_q)

In [ ]:
#print(new_q)
relative_entropy(p, new_q,  orders_masked.size)

In [ ]:
vals = convert_to_array(get_stat(orders_stats_original, "o_orderdate", "most_common_vals"))
freqs = convert_to_array(get_stat(orders_stats_original, "o_orderdate", "most_common_freqs"), True)

p = pd.Series(freqs, index = vals)

vals = convert_to_array(get_stat(orders_stats_masked, "o_orderdate", "most_common_vals"))
freqs = convert_to_array(get_stat(orders_stats_masked, "o_orderdate", "most_common_freqs"), True)

q = pd.Series(freqs, index = vals)

new_q = specification_prob(q,inverse_generalize_date_SO("MONTH"))

relative_entropy(p, new_q, new_q.size)